- 01 고객 3,500명에 대한 학습용 데이터(x_train.csv,y_train.csv)를 이용하여 성별예측모형을 만든 후,
- 이를 평가용 데이터(x_test.csv)에 적용하여 얻은 2482명의 고객 성별 예측값(남자일 확률)을 아래 형식의 csv파일로 제출하시오.
- (제출할 모델의 성능은 ROC_AUC 평가지표에 따라 채점)
- 제출형태
- 변수명.to_csv('수험번호.csv',index=False)

In [23]:
import pandas as pd
x_train = pd.read_csv('data/x_train.csv', encoding='cp949')
x_test = pd.read_csv('data/x_test.csv', encoding='cp949')
y_train = pd.read_csv('data/y_train.csv', encoding='cp949')

# 1. 라이브러리 및 데이터 확인
import pandas as pd
import numpy as np
# print(x_train.shape)
# print(x_test.shape)
# print(y_train.shape)
# print(x_train.info())
# print(x_test.info())
# print(y_train.info()) # 환불금액 결측치
# object type
cust_id = x_test['cust_id'].copy()
x_train = x_train.drop(columns=['cust_id'])
x_test = x_test.drop(columns=['cust_id'])
y_train = y_train.drop(['cust_id'], axis=1)
# print(x_train.info())
# print(x_test.info())
# print(y_train.info())
# 2. 데이터 탐색(EDA)
# print(x_train.describe().T)
# print(x_test.describe().T)
# print(x_train.describe(include='object').T)
# print(x_test.describe(include='object').T)
# print(y_train.value_counts())
# 3. 데이터 전처리 및 분리
# 1)결측치 2)이상치 3)변수 처리
# print(x_train.isnull().sum())
# print(x_test.isnull().sum())
# print(y_train.isnull().sum())
med = x_train['환불금액'].median()
x_train['환불금액'] = x_train['환불금액'].fillna(med)
x_test['환불금액'] = x_test['환불금액'].fillna(med)
# print(x_train.isnull().sum())
# print(x_test.isnull().sum())
# print(y_train.isnull().sum())
x_train = pd.get_dummies(x_train, dtype='uint8')
x_test = pd.get_dummies(x_test, dtype='uint8')
# print(x_train.info())
# print(x_test.info())
# print(x_train.shape)
# print(x_test.shape)
x_test = x_test.reindex(columns=x_train.columns, fill_value=0)
# print(x_train.shape)
# print(x_test.shape)
# print(x_train.head(3))
# print(x_test.head(3))
# 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train['gender'],
                                                  test_size=0.2,
                                                  stratify=y_train['gender']
                                                 )
# print(x_train.shape)
# print(x_val.shape)
# print(y_train.shape)
# print(y_val.shape)

# 4. 모델링 및 성능평가
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_val)
#print(y_pred)

from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix
acc = accuracy_score(y_val, y_pred)
auc = roc_auc_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)
# print(acc)
# print(auc)
# print(cm)

y_result = model.predict(x_test)
y_result_prob = model.predict_proba(x_test)
# print(y_result[:5])
# print(y_result_prob[:5])
result = pd.DataFrame({'result':y_result,
                       'prob0':y_result_prob[:,0],
                       'prob1':y_result_prob[:,1]})
# print(result[:5])

# 5. 예측값 제출
pd.DataFrame({'cust_id':cust_id, 'gender':y_result_prob[:,0]}).to_csv('단축키2유형1.csv', index=False)
df = pd.read_csv('단축키2유형1.csv')
print(df.head(3))

   cust_id  gender
0     3500    0.34
1     3501    0.86
2     3502    0.71
